<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/main/SER/CNN/CRNNdeep_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import sys
sys.path.append("/content/drive/MyDrive/team_runner/colab/emoDB/CNN")
from kfold import kfold

In [ ]:
X_train, X_test, Y_train, Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_more.npy", allow_pickle=True)
X = np.concatenate((X_train, X_test), axis=0)
X = np.expand_dims(X, axis=-1)
Y = np.concatenate((Y_train, Y_test), axis=0)

print(X.shape)
print(Y.shape)

(911, 128, 128, 1)
(911,)


In [ ]:
def create_model():
    inputs = keras.Input(shape=(128, 128, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D()(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D()(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D()(conv3)

    trans = keras.layers.Permute((2,1,3))(pool3)     # 확실히 시간순으로 lstm에 학습을 안하면 overfitting만 된다. 정확도 50%
    reshape = keras.layers.Reshape((-1, 16*128))(trans) # 열 개수(freq축)*ch
    linear = keras.layers.Dense(512)(reshape)
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=False, dropout=0.5))(linear)

    fc = keras.layers.Dense(128)(lstm)
    drop = keras.layers.Dropout(0.5)(fc) # 없어도 77%는 나오네
    output = keras.layers.Dense(7)(drop)
    return keras.Model(inputs=inputs, outputs=output)

In [ ]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0     

In [ ]:
kfold(5, create_model, X, Y, 0.001, 100, 120, 10)

Learning started. k=5
<<fold 1>>
Epoch	loss		train acc	test acc
10  	0.9183  	0.6637  	0.5090
20  	0.4962  	0.8230  	0.5561
30  	0.2074  	0.9350  	0.6754
40  	0.2934  	0.9125  	0.6372
50  	0.0673  	0.9875  	0.7846
60  	0.0024  	1.0000  	0.8337
70  	0.0019  	1.0000  	0.8227
80  	0.0008  	1.0000  	0.8327
90  	0.0005  	1.0000  	0.8277
100  	0.0004  	1.0000  	0.8277
110  	0.0005  	1.0000  	0.8277
120  	0.0002  	1.0000  	0.8227
max train accuracy: 1.0000
max test accuracy: 0.8487
<<fold 2>>
Epoch	loss		train acc	test acc
10  	1.0762  	0.6365  	0.5627
20  	0.6596  	0.7664  	0.7098
30  	0.3461  	0.8957  	0.6954
40  	0.1554  	0.9588  	0.7348
50  	0.0334  	0.9975  	0.7559
60  	0.0061  	1.0000  	0.7680
70  	0.0054  	1.0000  	0.7637
80  	0.0078  	0.9987  	0.7526
90  	0.0166  	1.0000  	0.7143
100  	0.0015  	1.0000  	0.7587
110  	0.0007  	1.0000  	0.7415
120  	0.0014  	1.0000  	0.7587
max train accuracy: 1.0000
max test accuracy: 0.7759
<<fold 3>>
Epoch	loss		train acc	test acc
10  	1.0056  	0.6144